# Train and deploy a model

We'll now delve into how to "manually" (leveraging the SDK and this Jupyter notebook) train a model in AzureML.

## Train a model

Initiate a connection to the Azure ML workspace and set up MLflow for tracking.

In [ ]:
## Train a model

# Handle to the workspace
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient
import mlflow

ml_client = MLClient.from_config(
    DefaultAzureCredential()
)

# Gather MLflow URI information from workspace
azureml_mlflow_uri = ml_client.workspaces.get(ml_client.workspace_name).mlflow_tracking_uri
mlflow.set_tracking_uri(azureml_mlflow_uri)

Import necessary libraries and set up the experiment in MLflow.

In [ ]:
# Import python packages
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
import numpy as np
import os

experiment_name = "Monitoring-Models-Experiment"
mlflow.set_experiment(experiment_name)

Load the dataset, convert it to a Pandas DataFrame, and prepare the directory for model saving.

In [ ]:
import mltable

# iterate over all versions of the data asset

data_asset = ml_client.data.get("diabetes-mltable-dev", label="latest") # You can also specify a specific version

tbl = mltable.load(data_asset.path)

df = tbl.to_pandas_dataframe()
df

model_path = "./models/monitoring"
os.makedirs(model_path, exist_ok=True)

Start logging the training process in MLflow, train a Decision Tree model, and log the model performance metrics.

In [ ]:
# delete model directory if it exists
import shutil
if os.path.exists(model_path):
    shutil.rmtree(model_path)

# Start Logging
mlflow.start_run()

# Enable autologging (optional)
# mlflow.sklearn.autolog()

diabetes = df

feature_cols = ['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']

# Breaking up data into input/target features
X, y = diabetes[feature_cols].values, diabetes['Diabetic'].values

# Breaking data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Training a model:
model = DecisionTreeClassifier().fit(X_train, y_train)

# Calculating performance and logging them
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
mlflow.log_metric('Accuracy', float(acc))

y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
mlflow.log_metric('AUC', float(auc))

Infer the model signature, register the model to the workspace, and save the model to a file.

In [ ]:
from mlflow.models import infer_signature

signature = infer_signature(X_test, y_hat)

model_name = "monitoring-diabetes"

# Registering the model to the workspace
print("Registering the model via MLFlow")
mlflow.sklearn.log_model(
    sk_model=model,
    registered_model_name=model_name,
    artifact_path="model",
    signature=signature
)

# Saving the model to a file
mlflow.sklearn.save_model(
    sk_model=model, 
    path=model_path,
    signature=signature
)

# Stop logging
mlflow.end_run()


We now also want to save the training data for model monitoring. We therefore use the same Version as the model version.

In [ ]:
import time
import mltable
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes
import pandas as pd

pd.DataFrame(X_train, columns=feature_cols).to_csv("./data/train/X_train.csv", index=False)

tbl_train = mltable.from_delimited_files(paths=[{"file": "./data/train/X_train.csv"}])
tbl_train.save("./data/train")

# Define the data asset
training_data = Data(
    path="./data/train/",
    type=AssetTypes.MLTABLE,
    description=f"Training data for model {model_name}",
    name="diabetes-mltable-train",
    version=time.strftime("%Y.%m.%d.%H%M%S", time.gmtime()),
)

# Create or update the data asset in AzureML
ml_client.data.create_or_update(training_data)

## Deploy a Model

After training and registering the model, it's time to deploy it. The easiest and most explaining way is to use the AzureML Studio UI (you can of course also leverage the SDK, using the commented code). 

Deployment is really easy with **MLFlow models**, because we know the signature and dependencies of the model and therefore don't need to define a scoring script and environment.

Under `Models` select the recently trained model `monitoring-diabetes` and select `Deploy` as a `Real-time endpoint`.

- This action will create a new `Deployment` on a new or existing `Endpoint` (You can deploy multiple models behind one endpoint).
- Keep the instances at 3 instances, so that we can add autoscaling later.
- Make sure to enable data collection (still in preview as of Nov '23).

> Note: The deployment will take **ten to fifteen minutes**, make sure to **grab a cup of coffee**. &#9749;

In [ ]:
# ## Deploy a Model

# # Name the model you registered earlier in the training script
# registered_model_name = "monitoring-diabetes"

# # Let's pick the latest version of the model
# latest_model_version = max(
#     [int(m.version) for m in ml_client.models.list(name=registered_model_name)]
# )

# from azure.ai.ml.entities import (
#     ManagedOnlineEndpoint,
#     ManagedOnlineDeployment,
#     Model,
# )

# import datetime

# # Creating a unique online endpoint name with current datetime to avoid conflicts
# online_endpoint_name = "monitor-diabetes-" + datetime.datetime.now().strftime("%m%d%H%M%f")

# # Create an online endpoint
# endpoint = ManagedOnlineEndpoint(
#   name=online_endpoint_name,
#   description="This is a diabetes classifier online endpoint",
#   auth_mode="key",
#   tags={
#       "training_dataset": "diabetes-data",
#       "model_type": "sklearn.DecisionTreeClassifier",
#       "purpose": "demonstration"
#   },
# )

# endpoint_lro = ml_client.begin_create_or_update(endpoint)

In [ ]:
# while endpoint_lro.status() in ["InProgress", "Updating"]:
#     print("Endpoint creation in progress...", end='', flush=True)
#     time.sleep(5)
# print("Endpoint status: ", endpoint_lro.status())

In [ ]:
# import time
# from azure.ai.ml.entities import DataCollector

# model = ml_client.models.get(name=registered_model_name, version=latest_model_version)

# notebook_deployment = ManagedOnlineDeployment(
#     name="notebook-deployment-1",
#     endpoint_name=online_endpoint_name,
#     model=model,
#     instance_type="Standard_F4s_v2",
#     instance_count=1,
#     app_insights_enabled=True,
#     # data_collector=DataCollector(
#     #     collections={"collections": ...},
#     # )
# )

# # try:
# deployment_lro = ml_client.online_deployments.begin_create_or_update(notebook_deployment)
# print("Creating notebook deployment on endpoint")
# # except Exception as e:
# #     print(e)
# #     print("Waiting 3 Minutes...")
# #     time.sleep(180)
# #     deployment_lro = ml_client.online_deployments.begin_create_or_update(notebook_deployment)
# #     print("Creating notebook deployment on endpoint")


In [ ]:
# while deployment_lro.status() in ["InProgress", "Updating"]:
#     print("\rDeployment creation in progress...", end='', flush=True)
#     time.sleep(5)

## Explore the endpoint

### 1) Test inference

Once the deploymet is complete go to `Endpoints`, select the newly deployed endpoint, and then select `Test`. 

Inference test data should be pre-populated. If for some reason it is not, use this JSON Data
```JSON
[11.0, 97.0, 89.0, 11.0, 23.0, 46.47006691, 1.476670289, 39.0],
[3.0, 108.0, 63.0, 45.0, 297.0, 49.37516891, 0.100979095, 46.0],
[9, 103, 78, 25, 304, 29.58219193, 1.282869847, 43]
```
or `notebooks/test.json` for scoring.

### 2) Check the logs

Now check the `Logs` (right next to `Test`). 
These logs provide valuable insights into the inference, especially when debugging model deployments.

> Make sure to select the correct Deployment when you deployed multiple models behind the endpoint.

### 3) Configure auto scaling


Lastly, there are a few options to auto scale our endpoint. 

First let's go back to `Details` and search for the correct Deployment (remember you can have multiple `Deployments` per `Endpoint`). 

1) Right of the Deployment name, you'll find a `Pen Icon` to edit the Deployment. Check Deployment scale type **`Target Utilization`** (e.g. 70%). This will enable us to define auto-scaling based on the desired CPU load.

2) If you need more control over your autoscaling, go back to the Deployment and check `Configure auto scaling`. You are forwarded to the Model Deployment in the Azure Portal, where you can configure `Custom autoscale`.

### Data Collection

Now after having scored some samples (from running the Tests before), head to `Data` within the Studio. 

You'll now find a few more datasets such as `<endpointname>-inputs` (features) and `<endpointname>-outputs` (predictions). Explore the data that is automatically collected.

### Monitoring the Endpoint

Finally go back to the `Endpoint`, scroll to the bottom and select `View metrics`. This will lead you to the `Azure Application Insights` instance that logs your workspace. 

Explore the metrics logged in your `Application Insights`. 

Be aware, some metrics and logs are only collected after you enabled `Diagnostic settings`.

## Monitoring: Dataset and Model Drift

Azure ML recently added the preview of Model and Data monitoring. We can now leverage this powerful feature via `Monitoring` in the AzureML Studio.

Add a new monitor via `+ Add` and follow the wizard. 
- Select the model and deployment we just trained and created.
- Under `Configure data assets` add the training data `diabetes-mltable-train` we registered a bit earlier.
- Under `Select monitoring signals` make sure to hit edit one of the signal by selecting `Diabetic` as the target column.

Create the monitor, grab another coffee, and chat about your learning.

## (Optional) Monitoring Drift: Bring your own data

If you are keen on exploring **how to bring your own production data** (that is not collected from an online endpoint) now is your time to understand how that works.

Remember, we registered a `diabetes-urifolder-production` dataset in the first notebook. This repository also contains a `preprocess component` (in the `components/preprocess_production_data` directory). 

You can use both to investigate how you could bring your own data, and (if you know how to register components) set up a monitor.

## Next steps

In the next notebook we'll learn more about Data Drift. Continue with the next notebook to **Create Synthentic Data** and then gather inference data that triggers data drift detection.